In [3]:
def task3():
    import tensorflow as tf

    from hops import tensorboard
    from hops import hdfs
    from tensorflow.examples.tutorials.mnist import input_data
    
    fashion_mnist = input_data.read_data_sets('data/fashion', one_hot=True, source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')
    
    # Helpers
    def weight_var(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_var(shape, value=0.1):
        initial = tf.constant(value, shape=shape)
        return tf.Variable(initial)
    
    def bias_var_z(shape):
        return tf.Variable(tf.zeros(shape))
    
    def layer(tensor, in_dim, out_dim, name, activation=tf.nn.sigmoid):
        weights = weight_var([in_dim, out_dim])
        biases = bias_var_z([out_dim])
        pre = tf.matmul(tensor, weights) + biases
        post = activation(pre)
        tf.summary.histogram('activations', post)
        return post
    
    # Hardcoded params
    num_ch = 1
    num_classes = 10
    image_height = image_width = 28
    layer_widths = [200, 100, 60, 30, 10]
    
    # 1. Define variables and placeholders
    p_keep = tf.placeholder(tf.float32)
    X = tf.placeholder(tf.float32, shape=[None, image_height, image_width, num_ch])
    Y_ = tf.placeholder(tf.float32, shape=[None, 10])

    XX = tf.reshape(X, [-1, image_height * image_width])
    
    # Convolutional layers
    # stride of (1,1,1,1) • patch of 5X5 • input depth/channels of 1 • output depth/channels of 4
    W1 = weight_var([5, 5, 1, 4])
    B1 = bias_var([4])
    Y1 = tf.nn.max_pool(
        tf.nn.dropout(
            tf.nn.relu(
                tf.nn.conv2d(XX, W1, strides=[1, 1, 1, 1], padding='SAME') + B1), 
            pkeep),
        ksize=[1, 1, 1, 1], 
        strides=[1, 1, 1, 1], 
        padding='SAME')

    
    # stride of (1,2,2,1) • patch of 5X5 • input depth/channels of 4 - previous output • output depth/channels of 8
    W2 = weight_var([5, 5, 4, 8])
    B2 = bias_var([8])
    Y2 = tf.nn.max_pool(
        tf.nn.dropout(
            tf.nn.relu(
                tf.nn.conv2d(Y1, W2, strides=[1, 2, 2, 1], padding='SAME') + B2), 
            pkeep),
        ksize=[1, 2, 2, 1], 
        strides=[1, 2, 2, 1], 
        padding='SAME')
    
    # stride of (1,2,2,1)• patch of 4X4 • input depth/channels of 8 - previous output• output depth/channels of 12
    W3 = weight_var([4, 4, 8, 12])
    B3 = bias_var([12])
    Y3 = tf.nn.max_pool(
        tf.nn.dropout(
            tf.nn.relu(
                tf.nn.conv2d(Y1, W2, strides=[1, 2, 2, 1], padding='SAME') + B3), 
            pkeep),
        ksize=[1, 2, 2, 1], 
        strides=[1, 2, 2, 1], 
        padding='SAME')
    
    # Densely connected layer
    # Must reshape output into flat vector
    YY3 = tf.reshape(Y3, [-1, Y3.shape[1] * Y3.shape[2] * Y3.shape[3]]) 
    W4 = tf.Variable(tf.truncated_normal([int(Y3.shape[1] * Y3.shape[2] * Y3.shape[3]), 200], stddev=0.1))
    B4 = bias_variable([200])
    Y4 = tf.nn.relu(tf.matmul(YY3, W4) + B4)
    
    # Readout layer
    W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
    B5 = tf.Variable(tf.zeros([10]))
    Y  = tf.matmul(Y4, W5) + B5
    
#     H1  = layer(XX,    784,          layer_widths[0], 'sigmoid-1', tf.nn.relu)
#     H2  = layer(H1, layer_widths[0], layer_widths[1], 'sigmoid-2', tf.nn.relu)
#     H3  = layer(H2, layer_widths[1], layer_widths[2], 'sigmoid-3', tf.nn.relu)
#     H4  = layer(H3, layer_widths[2], layer_widths[3], 'sigmoid-4', tf.nn.relu)
#     Y   = layer(H4, layer_widths[3], layer_widths[4], 'identity',  tf.identity)
    
#     if use_dropout:
#         H1d = tf.nn.dropout(H1, p_keep)
#         H2d = tf.nn.dropout(H2, p_keep)
#         H3d = tf.nn.dropout(H3, p_keep)
#         H4d = tf.nn.dropout(H4, p_keep)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_))
    correct_prediction =  tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("cost", cross_entropy)
    tf.summary.scalar("accuracy", accuracy)
    tf.summary.scalar('cross_entropy', cross_entropy)
    
    
    global_step = tf.Variable(0, trainable=False)
    dyn_lr = tf.train.exponential_decay(0.005, global_step, 10000, 0.90, staircase=True)
    lr = 0.005
    if use_dropout:
        lr = dyn_lr
    with tf.name_scope('train'):
        with tf.name_scope('adam_optimizer'):
            train_step_adam = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    
    
    logdir = tensorboard.logdir()
    summary_op = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(logdir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(logdir + '/test')
        
    def epochs(train, test, train_step, _pkeep, num_epochs=100, batch_size=100):
        sess.run(init)
        
        accuracies = []
        losses = []

        for epoch in range(100):
            for it in range(100):
                batch_xs, batch_ys = train.next_batch(batch_size)
                feed_dict =  {XX: batch_xs, Y_: batch_ys, p_keep: _pkeep}
                _, summary = sess.run([train_step, summary_op], feed_dict=feed_dict)
                train_writer.add_summary(summary, epoch * 100 + it)

            # Compute accuracy and loss every 100 rounds
            feed_dict = {XX: test.images, Y_: test.labels, p_keep: 1.0}
            summary, acc = sess.run([summary_op, accuracy], feed_dict=feed_dict)
            loss = sess.run(cross_entropy, feed_dict=feed_dict)

            accuracies.append(acc)
            losses.append(loss)
            test_writer.add_summary(summary, epoch * 100)
        return (accuracies, losses)

    acc, loss = epochs(fashion_mnist.train, fashion_mnist.test, train_step_adam, 0.6)

    train_writer.close()
    test_writer.close()
    
    print("Accuracy: {}".format(acc))
    print("Loss: {}".format(loss))
    



An error was encountered:
Invalid status code '404' from http://10.0.104.161:8998/sessions/1170 with error payload: "Session '1170' not found."


In [ ]:
from hops import tflauncher
from hops import tensorboard

tb_hdfs_dir = tflauncher.launch(spark, task3)

#tensorboard.visualize(spark, tb_hdfs_dir)